In [160]:
import json
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [161]:

def obter_dados(fonte, fonte_amigos=None):
    if fonte == 'json':
        with open('cursos.json') as f:
            dados = json.load(f)
    elif fonte == 'webhook':
        url = 'https://exemplo.com/api/dados'
        response = requests.get(url)
        dados = response.json()
    else:
        raise ValueError('Fonte de dados inválida')

    if fonte_amigos == 'json':
        with open('amigos.json') as f:
            amigos = json.load(f)
    elif fonte_amigos == 'webhook':
        url = 'https://exemplo.com/api/amigos'
        response = requests.get(url)
        amigos = response.json()
    else:
        amigos = []

    return dados, amigos

In [162]:
fonte = input('Digite a fonte dos dados (json ou webhook): ')
fonte_amigos = input('Digite a fonte dos amigos (json, webhook ou nenhum): ')
dados, amigos = obter_dados(fonte, fonte_amigos)
df = pd.DataFrame(columns=['id', 'nome', 'idade', 'curso_nome', 'curso_categoria', 'curso_avaliacao', 'amigo_id', 'amigo_nome', 'amigo_curso_nome', 'amigo_curso_avaliacao'])
rows = []
for usuario in dados['usuarios']:
    for curso in usuario['cursos']:
        row = {
            'id': usuario['id'],
            'nome': usuario['nome'],
            'idade': usuario['idade'],
            'curso_nome': curso['nome'],
            'curso_categoria': curso['categoria'],
            'curso_avaliacao': curso['avaliacao'],
            'amigo_id': None,
            'amigo_nome': None,
            'amigo_curso_nome': None,
            'amigo_curso_avaliacao': None
        }
        rows.append(row)
        if 'amigos' in usuario:
            for amigo in usuario['amigos']:
                for amigo_curso in amigo['cursos']:
                    row = {
                        'id': usuario['id'],
                        'nome': usuario['nome'],
                        'idade': usuario['idade'],
                        'curso_nome': curso['nome'],
                        'curso_categoria': curso['categoria'],
                        'curso_avaliacao': curso['avaliacao'],
                        'amigo_id': amigo['id'],
                        'amigo_nome': amigo['nome'],
                        'amigo_curso_nome': amigo_curso['nome'],
                        'amigo_curso_avaliacao': amigo_curso['avaliacao']
                    }
                    rows.append(row)
df = pd.DataFrame(rows)


In [163]:
df.drop_duplicates(inplace=True)

In [154]:
cursos_agrupados = df.groupby(['curso_nome', 'curso_categoria'])['curso_avaliacao'].mean().reset_index()

In [164]:
cursos_agrupados.rename(columns={'curso_avaliacao': 'curso_media_avaliacao'}, inplace=True)

In [165]:
df = df.merge(cursos_agrupados, on=['curso_nome', 'curso_categoria'], how='left')

In [166]:
df.sort_values(['nome', 'curso_media_avaliacao'], ascending=[True, False], inplace=True)